# Lekce 2

## Kontingenční tabulky (pivot)

Pojem `pivot` obvykle překládáme jako kontingenční tabulka. Jde o tabulku, která porovnává dvě různé tzv. kategorie - jedna kategorie je použitá pro sloupce a druhá pro řádky. Pomocí kontingenční tabulky můžeme například porovnat tržby obchodních manažerů v jednotlivých zemích.

Pro sestavení kontingenční tabulky použijeme metodu `pivot_table`. Potřebujeme nastavit následující parametry:
- Sloupec, který použijeme pro sloupce sloupcovou a řádkovou dimenzi.
- Sloupec, který využijeme pro hodnoty, a agregační funkci.

Pokud chceme porovnat tržby sales manažerů pro jednotlivé země, použijeme sloupce `sales_manager` a `country` pro sloupce a řádky. Do "vnitřku" tabulky vložíme hodnoty sloupce `contract_value` a protože chceme vidět celkovou hodnotu, jako funkci použijeme `numpy.sum`.

Abychom si výsledky výpočtů trochu zpřehlednili, můžeme vydělit hodnotu sloupce `contract_value` milionem.

In [2]:
import pandas

df_actual = pandas.read_csv("data/sales_actual.csv")
df_actual.head()

,date,company,country,contract_value,sales_manager
0,2021-11-09,Allard LLC Inc,Canada,374511.0,Jones Leon
1,2023-01-03,Asselin and Sons Group,Canada,170005.2,Jones Leon
2,2023-01-19,Audet Group Ltd,Canada,465253.2,Jones Leon
3,2022-07-14,Audet-Cyr Group,Canada,747392.1,Jones Leon
4,2021-02-16,Beauchamp Inc Group,Canada,140489.0,Jones Leon


Pro větší přehlednost převedeme hodnoty `contract_value` na miliony.

In [ ]:
df_actual["contract_value"] = df_actual["contract_value"] / 1_000_000

Zkusme nejprve vytvořit přehledovou tabulku pomocí standardní agregace. Provedeme agregaci podle sloupců `sales_manager` a `country` a použijeme agregační funkci `sum()` pro sloupec `contract_value`.

In [3]:
df_actual_aggregated = df_actual.groupby(["sales_manager", "country"])["contract_value"].sum()
df_actual_aggregated = pandas.DataFrame(df_actual_aggregated)
df_actual_aggregated

contract_value
sales_manager country                       
Jones Leon    Canada              44212759.7
              France              26063881.5
              Italy               18008563.0
              Netherlands         11606895.2
Morgan Sarah  Canada               6787855.4
              France               5074422.3
              Italy                4047312.1
              Netherlands          4009210.2
              USA                 23110862.4
              United Kingdom      38861613.5
Thomas Denise France              26344091.5
              Italy               26610211.1
              Netherlands         14261435.4
Watts Emily   USA                  6212289.5
              United Kingdom      10289671.7

Nyní si zkusíme zobrazit stejnou informaci pomocí kontingenční tabulky, použijeme tedy funkci `pivot_table`.

Pozor, nepleť si funkci `pivot_table` a `pivot`! Funkci `pivot` též vytváří kontingenční tabulku, provádí však jen polovinu toho, co `pivot_table`. Přeskládá hodnoty v tabulce, ale neprovádí agregaci. Lze ji tedy použít pouze v případě, že pro každou kombinaci hodnot sloupců máš maximálně jeden záznam.

Nastavujeme následující parametry:
- `data` = tabulka, která obsahuje zdrojová data,
- `index` = sloupec sloužící jako řádková dimenze,
- `columns` = sloupec sloužící jako sloupcová dimenze,
- `values` = sloupec, ze kterého se počítají hodnoty uvnitř tabulky,
- `aggfunc` = funkce, která je použitá k výpočtu hodnot uvnitř tabulky.

In [5]:
df_actual_pivot = pandas.pivot_table(data=df_actual, values="contract_value", index="country", columns="sales_manager", aggfunc=sum)
df_actual_pivot

sales_manager,Jones Leon,Morgan Sarah,Thomas Denise,Watts Emily
country,,,,
Canada,44212759.7,6787855.4,NaN,NaN
France,26063881.5,5074422.3,26344091.5,NaN
Italy,18008563.0,4047312.1,26610211.1,NaN
Netherlands,11606895.2,4009210.2,14261435.4,NaN
USA,NaN,23110862.4,NaN,6212289.5
United Kingdom,NaN,38861613.5,NaN,10289671.7


Často je zajímavé zobrazit si i celkový součet pro řádky a sloupce, ten získáme pomocí parametru `margins`.

In [6]:
df_actual_pivot_count = pandas.pivot_table(df_actual, values="contract_value", index="country", columns="sales_manager", aggfunc=sum, margins=True)
df_actual_pivot_count

sales_manager,Jones Leon,Morgan Sarah,Thomas Denise,Watts Emily,All
country,,,,,
Canada,44212759.7,6787855.4,NaN,NaN,51000615.1
France,26063881.5,5074422.3,26344091.5,NaN,57482395.3
Italy,18008563.0,4047312.1,26610211.1,NaN,48666086.2
Netherlands,11606895.2,4009210.2,14261435.4,NaN,29877540.8
USA,NaN,23110862.4,NaN,6212289.5,29323151.9
United Kingdom,NaN,38861613.5,NaN,10289671.7,49151285.2
All,99892099.4,81891275.9,67215738.0,16501961.2,265501074.5


Zkusme nyní změnit agregační funkci a uvažujme, že nás zajímá počet uzavřených kontraktů pro jednotlivé země a sales manažery. Můžeme využít například funkci `len()`, kterou známe z určování délky řetězců a seznamů.

In [7]:
pandas.pivot_table(df_actual, values="contract_value", index="country", columns="sales_manager", aggfunc=len, margins=True)

sales_manager,Jones Leon,Morgan Sarah,Thomas Denise,Watts Emily,All
country,,,,,
Canada,96.0,14.0,NaN,NaN,110
France,56.0,12.0,58.0,NaN,126
Italy,38.0,10.0,63.0,NaN,111
Netherlands,27.0,9.0,30.0,NaN,66
USA,NaN,51.0,NaN,11.0,62
United Kingdom,NaN,85.0,NaN,22.0,107
All,217.0,181.0,151.0,33.0,582


### Relativní hodnoty v kontingenční tabulce

Občas mohou být zajímavější relativní hodnoty než absolutní. Uvažujme například situaci, kdy chceme vidět, kolik procent celkových tržeb v jednotlivých zemích zajistili jednotliví obchodníci. Abychom tuto informaci získali, vydělíme tržby jednotlivých obchodníků v jednotlivých zemích tržbami v celé zemi. 

Vydělení provedeme pomocí metody `div`. Výpočet spočívá v tom, že vydělíme hodnoty v řádcích tabulky součtem příslušného řádku. Proto je nutné vytvořit kontingenční tabulku s nastavením parametru `margins=True`.

In [12]:
df_actual_pivot = pandas.pivot_table(df_actual, values="contract_value", index="country", columns="sales_manager", aggfunc=sum, margins=True)
df_actual_pivot_percentage = df_actual_pivot.div(df_actual_pivot.iloc[:,-1], axis=0)
df_actual_pivot_percentage

sales_manager,Jones Leon,Morgan Sarah,Thomas Denise,Watts Emily,All
country,,,,,
Canada,0.866906,0.133094,NaN,NaN,1.0
France,0.453424,0.088278,0.458298,NaN,1.0
Italy,0.370043,0.083165,0.546792,NaN,1.0
Netherlands,0.388482,0.134188,0.477330,NaN,1.0
USA,NaN,0.788144,NaN,0.211856,1.0
United Kingdom,NaN,0.790653,NaN,0.209347,1.0
All,0.376240,0.308440,0.253166,0.062154,1.0


V tabulce například vidíme, že 96 % tržeb v Kanadě zajistil Jones Leon, 49 % tržeb ve Francii Thomas Denise atd.

Můžeme se na data podívat i z opačného úhlu a podívat se na procentuální rozdělení tržeb obchodníků v rámci zemí. K tomu potřebujeme vydělit tržby obchodníků v jednotlivých zemí celkovými tržbami jednotlivých obchodníků.

In [13]:
df_actual_pivot_percentage = df_actual_pivot.div(df_actual_pivot.iloc[-1,:], axis=1)
df_actual_pivot_percentage

sales_manager,Jones Leon,Morgan Sarah,Thomas Denise,Watts Emily,All
country,,,,,
Canada,0.442605,0.082889,NaN,NaN,0.192092
France,0.260920,0.061965,0.391933,NaN,0.216505
Italy,0.180280,0.049423,0.395893,NaN,0.183299
Netherlands,0.116194,0.048958,0.212174,NaN,0.112533
USA,NaN,0.282214,NaN,0.376458,0.110445
United Kingdom,NaN,0.474551,NaN,0.623542,0.185127
All,1.000000,1.000000,1.000000,1.000000,1.000000


Z této tabulky vidíme, že Jones Leon zajistil třetinu svých tržeb v Kanadě a další třetinu ve Francii, Morgan Sara polovinu tržeb ve Spojeném království atd.

## Skupiny

Číselné sloupce často chceme rozdělit do skupin, například:

- zákazníky e-shopu podle počtu či hodnoty nákupů (příležitostní vs. věrní zákazníci atd.),
- univerzity dle hodnocení v žebříčku univerzit (nejlepší, skvělé, průměrné atd.),
- filmy dle hodnocení uživatelů (vynikající, průměrné, špatné atd.),
- státy dle průměrného platu (na bohaté, středně bohaté, chudé),
- studentům dle počtu bodů v testu udělíme známky.

Rozdělení můžeme provést pomocí funkce `cut`. V případě našich dat můžeme rozdělit kontrakty do několik skupin, abychom rozlišovali malé, střední a velké kontrakty. Zvolíme si například rozsahy skupin dle tabulky.

| `contract_value` | `label` |
|---|---|
| 0 až 300 000| small|
| 300 000 až 1 000 000 | medium |
| více než 1 000 000 | big |

Jako první parametr funkce `cut` zadáme sérii, ze které skupiny určujeme. Druhá skupina jsou hranice, které dělí jednotlivé skupiny. Protože máme 3 skupiny, potřebujeme 4 hranice. Jako poslední hranici můžeme zadat nějakou hodně vysokou částku (např. miliardu), případně můžeme použít výraz `inf`, což je zkratka anglického infinity - nekonečno. Řetězec `"inf"` převedeme na desetinné číslo pomocí funkce `float()`. Jako parametr `labels` přidáme seznam s názvy skupin.

In [27]:
df_actual["group"] = pandas.cut(df_actual["contract_value"], [0, 300_000, 1_000_000, float("inf")], labels=["small", "medium", "big"])
df_actual.head()

,date,company,country,contract_value,sales_manager,group
0,2021-11-09,Allard LLC Inc,Canada,374511.0,Jones Leon,medium
1,2023-01-03,Asselin and Sons Group,Canada,170005.2,Jones Leon,small
2,2023-01-19,Audet Group Ltd,Canada,465253.2,Jones Leon,medium
3,2022-07-14,Audet-Cyr Group,Canada,747392.1,Jones Leon,medium
4,2021-02-16,Beauchamp Inc Group,Canada,140489.0,Jones Leon,small


## Dobrovolné čtení na doma

### Teplotní mapa

I kontingenční tabulku můžeme prezentovat jako graf, hodí se například graf typu teplotní mapa (Heat Map). Ten vypadá podobně jako kontingenční tabulka, hodnoty jsou ale podbarveny barevnou škálou. Vygenerování grafu modulem `matplotlib` je poměrně náročné, proto je v tomto případně lepší použít "nadstavbu", kterou je modul `seaborn`.

In [ ]:
import seaborn as sns
sns.heatmap(df_actual_pivot, annot=True, fmt=".1f", linewidths=.5, cmap="YlGn")

# Cvičení

## Titanic

V souboru [titanic.csv](data/titanic.csv) najdeš informace o cestujících na zaoceánském parníku Titanic. Vytvoř kontingenční tabulku, která porovná závislost mezi pohlavím cestujícího (soupec `Sex`), třídou (sloupec `Pclass`), ve které cestoval, a tím, jesti přežil potopení Titanicu (sloupec `Survived`). Zkus spočítat relativní počet přeživších pro jednotlivé kombinace třídy a pohlaví.

Dále z dat vyfiltruj pouze cestující, kteří cestovali v první třídě. Využij metodu `cut` na rozdělení cestujících do věkových skupin (zkus vytvořit např. 4 skupiny, můžeš definovat hranice skupin tak, aby vznikly skupiny děti, teenageři, dospělí a senioři). Urči počet přeživších pro jednotlivé kombinace pohlavní a věkové skupiny.